#（自分のアカウントの）Googleドライブをマウント


In [1]:

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# どこのフォルダを分析するか？
*  classify_dir に指定




In [2]:

import os

base_dir = r"/content/drive/MyDrive/OffloadContest/"
dir_classify= base_dir+r"train_annotations_A"





#Check dir_classifyに何がはいっているか

*   PNGファイルが沢山あるはず？



In [3]:
files_classify = os.listdir(dir_classify)
files_classify.sort()
print(files_classify)


['A0000.png', 'A0001.png', 'A0002.png', 'A0003.png', 'A0004.png', 'A0005.png', 'A0006.png', 'A0007.png', 'A0008.png', 'A0009.png', 'A0010.png', 'A0011.png', 'A0012.png', 'A0013.png', 'A0014.png', 'A0015.png', 'A0016.png', 'A0017.png', 'A0018.png', 'A0019.png', 'A0020.png', 'A0021.png', 'A0022.png', 'A0023.png', 'A0024.png', 'A0025.png', 'A0026.png', 'A0027.png', 'A0028.png', 'A0029.png', 'A0030.png', 'A0031.png', 'A0032.png', 'A0033.png', 'A0034.png', 'A0035.png', 'A0036.png', 'A0037.png', 'A0038.png', 'A0039.png', 'A0040.png', 'A0041.png', 'A0042.png', 'A0043.png', 'A0044.png', 'A0045.png', 'A0046.png', 'A0047.png', 'A0048.png', 'A0049.png', 'A0050.png', 'A0051.png', 'A0052.png', 'A0053.png', 'A0054.png', 'A0055.png', 'A0056.png', 'A0057.png', 'A0058.png', 'A0059.png', 'A0060.png', 'A0061.png', 'A0062.png', 'A0063.png', 'A0064.png', 'A0065.png', 'A0066.png', 'A0067.png', 'A0068.png', 'A0069.png', 'A0070.png', 'A0071.png', 'A0072.png', 'A0073.png', 'A0074.png', 'A0075.png', 'A0076.png'

# 準備

In [4]:
from PIL import Image
import numpy as np

# グラフの準備

In [5]:
import plotly
import plotly.graph_objs as go
import plotly.io as pio
import plotly.express as px
pio.renderers.default = "colab"
plotly.__version__

'4.4.1'

# 資料を基に、PixelをRGBパレットに従って、クラス分け、クラスに応じた文字列を定義

In [6]:
definitions = [[255,0,255,'OK/Able to drive(2)'],
  [128,64,128,"OK/road(19)"],
  [255,128,128,"OK/dirt road(19)"],
  [190,153,153,"OK/parking lot(19)"],
  [102,102,156,"OK/driveway(19)"],
  [152,251,152,"OK/other road marking(19)"],
  [255,0,0,"OK/rail(19)"],
  [0,60,100,"OK/lane boundary(19)"],
  [0,80,100,"OK/other lane marking(19)"],
  [120,240,120,"OK/puddle(19)"],
  [128,0,255,"OK/rut(19)"],
  [255,0,0,"NG/Unable to drive(2)"],
  [0,0,70,"NG/other obstacle(19)"],
  [70,130,180,"NG/sky(19)"],
  [220,20,60,"NG/person(19)"],
  [119,11,32,"NG/two-wheel vehicle(19)"],
  [0,0,142,"NG/car(19)"],
  [220,220,0,"NG/traffic sign(19)"],
  [70,70,70,"NG/building(19)"],
  [90,120,130,"NG/crack(19)"],
  [255,255,255,"NG/snow(19)"],
  [-1,-1,-1,"Undefined"]]


def eq( rgb, def_i):
  return (rgb[0]==def_i[0] and rgb[1]==def_i[1] and rgb[2]==def_i[2])

def rgb_to_str( rgb ):
  if (rgb is None):
    return "Undefined"
  for def_item in definitions:
    if (eq(rgb, def_item)):
      return def_item[3]
  return "Undefined"




# フォルダ内の全ファイルを処理する


In [8]:
global_labels= [ i[3] for i in definitions]
global_label_pix_counters = [0] * len(global_labels)
global_label_img_counters = [0] * len(global_labels)


for fn in files_classify[]:
  print("Process.."+fn)

  img_fn=os.path.join(dir_classify,fn)

  # 変換後の画像をPILで読み込み
  pil_img = Image.open(img_fn)

  #インデックスカラーに自動変換
  pil_img_indexed = pil_img.convert("P",palette=Image.ADAPTIVE)

  #カラーパレットを取得
  palette = pil_img_indexed.getpalette()

  #カラーパレット表示
  #print(palette[:18])

  #[R, G, B, R, G, B, ...]　-> [RGB][RGB]....に変形
  palette_rgb = np.array(palette).reshape(-1, 3)
  #print(palette_rgb[:6])

  # 実データ（ピクセル値がインデックスに変換されている）
  img_data = pil_img_indexed.getdata()
  # ピクセルをNumpy Arrayに変換
  index_list = np.array(list(map(int, img_data)))
  # 一意のリストと総数をカウント
  unique_idx_list, counts = np.unique(index_list, return_counts=True)

  #どこのdefinitionのカウンターを使うか？
  for pixel_idx in unique_idx_list:
    n= global_labels.index( rgb_to_str(palette_rgb[pixel_idx]) )
    #どこのdefinitionのカウンターを使うか？
    print( "{0} --> {1}".format(pixel_idx,n))
    #ラベルをピクセル数で積算
    global_label_pix_counters[n] += counts[pixel_idx]
    #ラベルを＋１する
    global_label_img_counters[n] += 1

  

Process..A0000.png
0 --> 2
1 --> 13
2 --> 12
Process..A0001.png
0 --> 2
1 --> 12
Process..A0002.png
0 --> 2
1 --> 13
2 --> 12
Process..A0003.png
0 --> 2
1 --> 13
2 --> 12
Process..A0004.png
0 --> 17
1 --> 13
2 --> 2
3 --> 18
4 --> 1
5 --> 14
6 --> 7
7 --> 16
8 --> 12
Process..A0005.png
0 --> 2
1 --> 13
2 --> 12
Process..A0006.png
0 --> 2
1 --> 13
2 --> 12
Process..A0007.png
0 --> 2
1 --> 13
2 --> 16
3 --> 12
Process..A0008.png
0 --> 2
1 --> 13
2 --> 12
Process..A0009.png
0 --> 2
1 --> 1
2 --> 13
3 --> 12
Process..A0010.png
0 --> 2
1 --> 13
2 --> 14
3 --> 12
Process..A0011.png
0 --> 2
1 --> 13
2 --> 12
Process..A0012.png
0 --> 2
1 --> 13
2 --> 12
Process..A0013.png
0 --> 2
1 --> 13
2 --> 12
Process..A0014.png
0 --> 2
1 --> 13
2 --> 12
Process..A0015.png
0 --> 2
1 --> 13
2 --> 12
Process..A0016.png
0 --> 13
1 --> 1
2 --> 7
3 --> 12
Process..A0017.png
0 --> 2
1 --> 1
2 --> 13
3 --> 18
4 --> 7
5 --> 16
6 --> 12
Process..A0018.png
0 --> 2
1 --> 13
2 --> 12
Process..A0019.png
0 --> 2
1 --> 1

KeyboardInterrupt: ignored

In [9]:
# クラス分けピクセル数の積算
fig_global_pix = dict({
    "data": [{"type": "bar",
              "x": global_labels,
              "y": global_label_pix_counters}],
    "layout": {"title": {"text": "Classify Pixel Count:"+dir_classify}}
})

pio.show(fig_global_pix)

In [10]:
# クラス分け画像数の積算
fig_global_img = dict({
    "data": [{"type": "bar",
              "x": global_labels,
              "y": global_label_img_counters}],
    "layout": {"title": {"text": "Classify Image Count:"+dir_classify}}
})

pio.show(fig_global_img)